In this project, we construct an enhanced index using black-litterman model

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy.optimize import minimize

## Data Source and preprocess

The data we used in this project include:
1. The latest (as of November 30, 2023) CSI 300 Index Constituents Information
2. Monthly return of stocks listed in CSI300 Index from October 1, 2016 to November 30, 2023
3. Monthly market return of Shanghai and Shenzhen A-share markets (total market cap weighted, dividends reinvested) from October 1, 2016 to November 30, 2023 
4. Monthly risk free rate (one-year deposite rate) from October 1, 2016 to November 30, 2023 
5. Market capitalization of stocks listed in CSI300 (as of September 30, 2023)

The data are downlaoded from CSMAR database.

In [2]:
# load data
df_rf = pd.read_csv("risk_free_rate.csv")
df_cap = pd.read_csv('cap.csv')
df_mo_ret = pd.read_csv("mo_ret.csv")
df_mkt = pd.read_csv("mkt_ret.csv")

In [3]:
# preprocess data 
df_rf['date'] = pd.to_datetime(df_rf['date'])
df_rf['ym'] = df_rf['date'].dt.to_period('M')# from year-month-day to year-month
df_rf.drop('date',axis=1,inplace=True)
df_rf = df_rf.drop_duplicates('ym',keep='first')
df_rf.head()

,monthly rf (%),ym
0,0.1241,2016-10
31,0.1241,2016-11
61,0.1241,2016-12
92,0.1241,2017-01
123,0.1241,2017-02


In [4]:
df_cap.drop('date',axis=1,inplace=True)
df_cap.head()

,Stkcd,Market cap
0,1,2.173420e+11
1,2,1.559550e+11
2,63,1.562930e+11
3,69,2.984881e+10
4,100,7.384205e+10


In [5]:
df_mo_ret.head()

,Stkcd,date,Mret
0,1,Oct-16,0.008820
1,1,Nov-16,0.043716
2,1,Dec-16,-0.047120
3,1,Jan-17,0.025275
4,1,Feb-17,0.016077


In [6]:
df_mkt = df_mkt[df_mkt['Markettype']==5].drop('Markettype',axis=1)
df_mkt.head()

,date,return
1,Oct-16,0.031460
20,Nov-16,0.041749
39,Dec-16,-0.049932
57,Jan-17,0.005073
76,Feb-17,0.035021


In [7]:
# merge monthly stock return data and market return data
merged_data = pd.merge(df_mo_ret, df_mkt, on='date', how='left')

# if the stock doesn't have full data for the whole period, then drop the stock
merged_data['date'] = pd.to_datetime(merged_data['date'],format='%b-%y')
start_date = pd.to_datetime('2016-10')
end_date = pd.to_datetime('2023-11')

mask = (merged_data['date'] >= start_date) & (merged_data['date'] <= end_date)
merged_data = merged_data[mask]

# combine with capitalization data
merged_data = pd.merge(merged_data, df_cap, on='Stkcd', how='left')

# combine with risk free rate
merged_data['ym'] = merged_data['date'].dt.to_period('M')
merged_data.drop('date',axis=1,inplace=True)
merged_data = pd.merge(merged_data, df_rf,on='ym',how='left')

# calculate the excess return of stock and excess return of market
merged_data['excess market return'] = merged_data['return'] - merged_data['monthly rf (%)']/100
merged_data['excess stock return'] = merged_data['Mret'] - merged_data['monthly rf (%)']/100

merged_data.head()

,Stkcd,Mret,return,Market cap,ym,monthly rf (%),excess market return,excess stock return
0,1,0.008820,0.031460,2.173420e+11,2016-10,0.1241,0.030219,0.007579
1,1,0.043716,0.041749,2.173420e+11,2016-11,0.1241,0.040508,0.042475
2,1,-0.047120,-0.049932,2.173420e+11,2016-12,0.1241,-0.051173,-0.048361
3,1,0.025275,0.005073,2.173420e+11,2017-01,0.1241,0.003832,0.024034
4,1,0.016077,0.035021,2.173420e+11,2017-02,0.1241,0.033780,0.014836


In [8]:
# split data into train data and test data
time = pd.to_datetime('2023-8-1')
merged_data_test =  merged_data[merged_data['ym']>time.to_period('M')].copy()
merged_data_train = merged_data[merged_data['ym']<=time.to_period('M')].copy()
merged_data_train.head()

,Stkcd,Mret,return,Market cap,ym,monthly rf (%),excess market return,excess stock return
0,1,0.008820,0.031460,2.173420e+11,2016-10,0.1241,0.030219,0.007579
1,1,0.043716,0.041749,2.173420e+11,2016-11,0.1241,0.040508,0.042475
2,1,-0.047120,-0.049932,2.173420e+11,2016-12,0.1241,-0.051173,-0.048361
3,1,0.025275,0.005073,2.173420e+11,2017-01,0.1241,0.003832,0.024034
4,1,0.016077,0.035021,2.173420e+11,2017-02,0.1241,0.033780,0.014836


## Test the existence of low risk anomaly 

Low risk anomaly indicates that the stock with high CAPM beta tends to have lower return compared to those with low CAPM beta in the stock market. 

In this section, we first calculate beta for each stock and sort the stocks into six groups based on value of beta. Then we calculate the portfolio excess return and volatility for each group of stocks. Finally we calculate sharpe ratio for each portfolio by using excess return and the historical volatility.

The formula for CAPM model is:
$E(R_i)-R_f = \beta_i * (E(R_m)-R_f)$

The formula for sharpe ratio is:

$D_t = R_t - R_{ft}$

Sharpe ratio = $\frac{\bar D_t}{\sigma (D_t)} $, where $\bar D_t$ is the expected return and $\sigma (D_t)$ is the standard deviation.

In [9]:
# calculate beta of each stock
beta_values = []
for stkcd, group in merged_data_train.groupby('Stkcd'):
    if len(group) > 1: #ensure there is enough data for regression
        X = sm.add_constant(group['excess market return'])
        y = group['excess stock return']
        model = sm.OLS(y, X, missing='drop').fit() #drop if there is a missing data when regressing
        beta = model.params['excess market return'] if 'excess market return' in model.params else np.nan
    else:
        beta = np.nan
        
    beta_values.extend([beta]*len(group))

merged_data_train['beta'] = beta_values
merged_data_train.head()

,Stkcd,Mret,return,Market cap,ym,monthly rf (%),excess market return,excess stock return,beta
0,1,0.008820,0.031460,2.173420e+11,2016-10,0.1241,0.030219,0.007579,0.997797
1,1,0.043716,0.041749,2.173420e+11,2016-11,0.1241,0.040508,0.042475,0.997797
2,1,-0.047120,-0.049932,2.173420e+11,2016-12,0.1241,-0.051173,-0.048361,0.997797
3,1,0.025275,0.005073,2.173420e+11,2017-01,0.1241,0.003832,0.024034,0.997797
4,1,0.016077,0.035021,2.173420e+11,2017-02,0.1241,0.033780,0.014836,0.997797


In [10]:
# calculate the beta quantile
group_num = 6 # divide the data into 6 groups
beta_quantile = merged_data_train[['Stkcd','beta']].drop_duplicates('Stkcd')
beta_quantile['beta_quantile'] = pd.qcut(beta_quantile['beta'], q=group_num, labels=False, duplicates='drop')
beta_quantile = beta_quantile[['Stkcd', 'beta_quantile']]

# merge the merged data and beta quantile
merged_data_train = pd.merge(merged_data_train, beta_quantile, on='Stkcd',how='left')

# calculate monthly excess return of portfolio
p_ret = merged_data_train.groupby(['beta_quantile', 'ym'])['excess stock return'].mean().reset_index()
p_ret.rename(columns={'excess stock return':'excess portfolio return'},inplace=True)

p_ret.head()

,beta_quantile,ym,excess portfolio return
0,0,2016-10,0.015380
1,0,2016-11,0.035964
2,0,2016-12,-0.022851
3,0,2017-01,0.028162
4,0,2017-02,0.011472


In [11]:
# calculate the excess expected return and historical volatility of portfolio
ret = p_ret.groupby(['beta_quantile'])['excess portfolio return'].mean().reset_index()
ret.rename(columns={'excess portfolio return':'expected excess return'},inplace=True)

vol = p_ret.groupby(['beta_quantile'])['excess portfolio return'].std().reset_index()
vol.rename(columns={'excess portfolio return':'volatility'},inplace=True)
p_sum = pd.merge(ret,vol,on='beta_quantile',how='left')

# calculate sharpe ratio, which is annualized by multiplying sqrt(12)
p_sum['sharpe ratio'] = np.sqrt(12) * p_sum['expected excess return'] / p_sum['volatility']
p_sum

,beta_quantile,expected excess return,volatility,sharpe ratio
0,0,0.007351,0.027990,0.909724
1,1,0.011312,0.037243,1.052192
2,2,0.010587,0.046792,0.783750
3,3,0.014699,0.056104,0.907582
4,4,0.017127,0.068490,0.866270
5,5,0.022256,0.089168,0.864634


Notice that the expected excess return and sharpe ratio of group 1 is higher than group 2. Thus we confirm that there is a low risk anomaly existed in the Chinese A share stock market as of November 30, 2023.

## Construct Black-Litterman Model 

In [12]:
# calculate the weights of each portfolio based on its market capitalization
df_unique = merged_data_train.drop_duplicates('Stkcd',keep='first')
p_cap = df_unique.groupby(['beta_quantile'])['Market cap'].sum().reset_index()
total_cap = p_cap['Market cap'].sum()
p_cap['weight'] = p_cap['Market cap']/total_cap

# calculate the covariance matrix of portfolio excess return
# construct a new dataframe
df_new = pd.DataFrame()
for i in range(6):
    df_temp = p_ret.loc[p_ret['beta_quantile'].isin([i])]
    df_new[i] = df_temp['excess portfolio return'].values
# calculate the covariance
Sigma = np.array(df_new.cov())

def black_litterman(market_weights, cov_matrix, gamma=2, tau=1): # gamma represents degree of risk aversion
    # implied return
    miu_0 = gamma  * cov_matrix @ market_weights
    
    # construct view
    P = np.array([[0,1,-1,0,0,0]]).T # view matrix, we long group 1 and short group 2 
    period = 24 # use data over the latest 2 years to construct view
    diff = np.array(df_new[1]-df_new[2])[-period:].mean() 
    Q = np.array([diff])
    
    # conviction of view
    Omega = np.array(df_new[1]-df_new[2])[-period:].var()
    # return posterior expected return
    return np.linalg.inv(np.linalg.inv(tau*cov_matrix)+(1/Omega)*P@P.T)\
           @ (np.linalg.inv(tau*cov_matrix)@miu_0+(1/Omega)*P@Q)
    
miu_p = black_litterman(p_cap['weight'].values, Sigma)
miu_p

array([0.00144739, 0.00211615, 0.00021265, 0.00113351, 0.00055594,
       0.00113862])

In [13]:
def optimize_portfolio(returns, cov_matrix, gamma=2):
    def obj(weights):
        portfolio_ret = weights.T @ returns
        portfolio_var = weights.T @ cov_matrix @ weights
        return -(portfolio_ret - 0.5 * gamma * portfolio_var)
    
    constraints = [{'type':'eq','fun':lambda x: np.sum(x)-1}] #sum of weights equals to 1
    bounds = [(0,1) for _ in range(len(returns))] # no short position
    
    x0 = np.ones(len(returns)) / len(returns) # set the initial value
    result = minimize(obj,x0,method='SLSQP',bounds=bounds,constraints=constraints)
    return result.x
    
optimal_weights = optimize_portfolio(miu_p,Sigma)
optimal_weights

array([5.37369538e-01, 4.62630462e-01, 6.71121145e-17, 0.00000000e+00,
       0.00000000e+00, 5.83842870e-17])

In [14]:
# compare the performance of portfolio with optimal weights and portfolio with equal weights
merged_data_test = pd.merge(merged_data_test,beta_quantile,on='Stkcd',how='left')
p_ret_test = merged_data_test.groupby(['beta_quantile','ym'])['excess stock return'].mean().reset_index()
p_ret_test.rename(columns={'excess stock return':'excess portfolio return'},inplace=True)
p_ret_test.head()

,beta_quantile,ym,excess portfolio return
0,0,2023-09,-0.006342
1,0,2023-10,-0.019766
2,0,2023-11,0.007751
3,1,2023-09,-0.010188
4,1,2023-10,-0.036333


In [15]:
month = p_ret_test['ym'].unique()

optimal_arr=[]
equal_arr=[]
original_arr=[]
equal_weights = np.ones(group_num)/group_num

for i in month:
    df_temp = p_ret_test[p_ret_test['ym']==i]
    opt =  np.array(df_temp['excess portfolio return']).T @ optimal_weights 
    eql = np.array(df_temp['excess portfolio return']).T @ equal_weights 
    ori = np.array(df_temp['excess portfolio return']).T @ p_cap['weight'].values # cap weight
    
    optimal_arr.append(opt)
    equal_arr.append(eql)
    original_arr.append(ori)
    
optimal_arr = np.array(optimal_arr)
equal_arr = np.array(equal_arr)
original_arr = np.array(original_arr)

print('Average return for portfolio with optimal weights:',optimal_arr.mean())
print('Average return for portfolio with equal weights:',equal_arr.mean())
print('Average return for portfolio with captalization weights:',original_arr.mean(),"\n")

print('Accumulate return for portfolio with optimal weights:',np.prod(optimal_arr+1))
print('Accumulate return for portfolio with equal weights:',np.prod(equal_arr+1))
print('Accumulate return for portfolio with captalization weights:',np.prod(original_arr+1))

Average return for portfolio with optimal weights: -0.0098480529343012
Average return for portfolio with equal weights: -0.021759643333333332
Average return for portfolio with captalization weights: -0.01915639360819402 

Accumulate return for portfolio with optimal weights: 0.970466373040794
Accumulate return for portfolio with equal weights: 0.9360486189950115
Accumulate return for portfolio with captalization weights: 0.9435112965754859
